In [2]:
import pandas as pd
import numpy as np

# Load the one-minute stock prices CSV file (adjust the path as needed)
file_name = "stock_prices_1m"
df = pd.read_csv(f'{file_name}.csv')

# If a Date column exists, parse it as datetime and sort by time
if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values('Date', inplace=True)
    df.reset_index(drop=True, inplace=True)
    

#############################################
# 1. MACD Calculation
#############################################
# MACD is computed using two EMAs of the closing price.
# Typical periods: 12 and 26 for EMAs and 9 for the signal line.
df['EMA12'] = df['Close'].ewm(span=12, adjust=False).mean()
df['EMA26'] = df['Close'].ewm(span=26, adjust=False).mean()
df['MACD'] = df['EMA12'] - df['EMA26']
df['Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
df['Hist'] = df['MACD'] - df['Signal']

#############################################
# 2. RSI Calculation (Period = 14)
#############################################
delta = df['Close'].diff()
gain = delta.copy()
loss = delta.copy()
gain[gain < 0] = 0
loss[loss > 0] = 0
loss = loss.abs()

window_length = 14
avg_gain = gain.rolling(window=window_length, min_periods=window_length).mean()
avg_loss = loss.rolling(window=window_length, min_periods=window_length).mean()
rs = avg_gain / avg_loss
df['RSI'] = 100 - (100 / (1 + rs))

#############################################
# 3. KDJ Calculation (Period = 9)
#############################################
# Compute the Raw Stochastic Value (RSV)
period = 9
df['Low_Min'] = df['Low'].rolling(window=period, min_periods=period).min()
df['High_Max'] = df['High'].rolling(window=period, min_periods=period).max()
df['RSV'] = (df['Close'] - df['Low_Min']) / (df['High_Max'] - df['Low_Min']) * 100

# Initialize K, D, and J. Typically start K and D at 50.
df['K'] = np.nan
df['D'] = np.nan
df['J'] = np.nan

for i in range(len(df)):
    if np.isnan(df.loc[i, 'RSV']):
        continue
    if i == df['RSV'].first_valid_index():
        df.loc[i, 'K'] = 50
        df.loc[i, 'D'] = 50
    else:
        df.loc[i, 'K'] = (2/3) * df.loc[i-1, 'K'] + (1/3) * df.loc[i, 'RSV']
        df.loc[i, 'D'] = (2/3) * df.loc[i-1, 'D'] + (1/3) * df.loc[i, 'K']
    df.loc[i, 'J'] = 3 * df.loc[i, 'K'] - 2 * df.loc[i, 'D']

#############################################
# 4. OSC Calculation: Price Oscillator
#############################################
# Using the percentage difference between 5-minute and 10-minute SMAs.
sma_fast = df['Close'].rolling(window=5).mean()
sma_slow = df['Close'].rolling(window=10).mean()
df['OSC'] = (sma_fast - sma_slow) / sma_slow * 100

#############################################
# 5. BOLL Calculation: Bollinger Bands
#############################################
# Bollinger Bands using a 20-period SMA and 2 standard deviations.
boll_period = 20
df['BOLL_Mid'] = df['Close'].rolling(window=boll_period).mean()
df['BOLL_STD'] = df['Close'].rolling(window=boll_period).std()
df['BOLL_Upper'] = df['BOLL_Mid'] + 2 * df['BOLL_STD']
df['BOLL_Lower'] = df['BOLL_Mid'] - 2 * df['BOLL_STD']

#############################################
# 6. BIAS Calculation: Price Bias Indicator
#############################################
# BIAS is the percentage difference between the closing price and its 6-period SMA.
bias_period = 6
sma_bias = df['Close'].rolling(window=bias_period).mean()
df['BIAS'] = (df['Close'] - sma_bias) / sma_bias * 100

#############################################
# Display or save results
#############################################
# Print the last few rows to check the calculated indicators
print(df.tail())

# Optionally, save the results to a new CSV file
df.to_csv(f'{file_name}_with_indicators.csv', index=False)



                        Datetime        Open        High         Low  \
13645  2025-03-12 19:59:00+00:00  167.169998  167.270004  167.050003   
13646  2025-03-12 19:59:00+00:00  619.739990  619.919983  619.320007   
13647  2025-03-12 19:59:00+00:00  383.369995  383.609985  383.250000   
13648  2025-03-12 19:59:00+00:00  115.730003  115.930000  115.669998   
13649  2025-03-12 19:59:00+00:00  247.865005  248.279999  247.779999   

            Close   Volume Ticker       EMA12       EMA26       MACD  ...  \
13645  167.080002   445690  GOOGL  244.126724  261.545185 -17.418461  ...   
13646  619.340027   157299   META  301.851848  288.048507  13.803341  ...   
13647  383.279999   457654   MSFT  314.379255  295.102692  19.276564  ...   
13648  115.739998  3543537   NVDA  283.819370  281.816566   2.002804  ...   
13649  248.279999  1301638   TSLA  278.351774  279.332376  -0.980602  ...   

             RSV          K          D          J        OSC    BOLL_Mid  \
13645  10.186838  18.991540 